In [54]:
import pandas as pd
import seaborn as sns
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt

from torch_geometric.data import Data
import warnings
warnings.filterwarnings("ignore")
import torch


In [55]:
import torch
print("CUDA Available:", torch.cuda.is_available())
x = torch.tensor([1.0, 2.0, 3.0], device='cuda')
print(x)

CUDA Available: True
tensor([1., 2., 3.], device='cuda:0')


In [56]:
data=pd.read_csv("data_consolidation_standardization.csv")
data=data.drop(labels="Unnamed: 0", axis=1)
# data_test=data.loc[data['CellID']<=100]

In [57]:
data.sort_values(['CellID','datetime'], inplace=True)
data_test=data

In [58]:
data_test

,datetime,CellID,internet,calls,sms,longitude,latitude
0,2013-11-01 00:00:00,1,-1.045031,-0.739046,-0.846748,9.160012,45.358657
100,2013-11-01 01:00:00,1,-1.074765,-0.750493,-0.865873,9.160012,45.358657
200,2013-11-01 02:00:00,1,-1.087136,-0.754285,-0.873567,9.160012,45.358657
300,2013-11-01 03:00:00,1,-1.096080,-0.757753,-0.875403,9.160012,45.358657
400,2013-11-01 04:00:00,1,-1.109877,-0.756706,-0.878479,9.160012,45.358657
...,...,...,...,...,...,...,...
16399,2013-11-07 19:00:00,100,-0.050505,-0.004918,-0.064944,9.160606,45.568069
16499,2013-11-07 20:00:00,100,-0.141303,-0.239126,-0.156799,9.160606,45.568069
16599,2013-11-07 21:00:00,100,-0.231419,-0.426115,-0.262352,9.160606,45.568069
16699,2013-11-07 22:00:00,100,-0.319378,-0.600432,-0.395659,9.160606,45.568069


# 重构数据

In [59]:
# 初始化一个数组来存储重构后的数据
num_nodes = 100
num_timesteps = 168
num_features = 3

X为需要用到的数据

In [60]:
# 创建一个空的 numpy 数组，用于存储每个节点的特征，这里是提取和时序有关的数据，为训练集和测试集构建做准备
X = np.zeros((num_nodes, num_timesteps, num_features))

# 遍历每个基站，填充数据
for i in range(num_nodes):
    # 提取当前基站的数据
    node_data = data_test[data_test['CellID'] == i+1]  # 假设 CellID 从 1 开始
    # 确保数据的长度正确
    if len(node_data) == num_timesteps:
        X[i, :, 0] = node_data['internet'].values
        X[i, :, 1] = node_data['calls'].values
        X[i, :, 2] = node_data['sms'].values
    else:
        print(f"Warning: Node {i+1} has incorrect number of timesteps: {len(node_data)}")


In [61]:
X.shape

(100, 168, 3)

In [62]:
X#X为100个节点3个特征168个时序的数据

array([[[-1.0450308 , -0.73904563, -0.84674845],
        [-1.07476506, -0.75049338, -0.86587277],
        [-1.08713609, -0.7542851 , -0.87356665],
        ...,
        [-0.96361034, -0.67310022, -0.74354086],
        [-0.97581032, -0.71657865, -0.76440385],
        [-1.00971515, -0.73868004, -0.80597322]],

       [[-1.02265791, -0.73796843, -0.84425615],
        [-1.05836134, -0.74950306, -0.86310406],
        [-1.06950522, -0.75192882, -0.87142658],
        ...,
        [-0.92947228, -0.66630709, -0.72315621],
        [-0.93924703, -0.7134834 , -0.74826225],
        [-0.97507457, -0.73674196, -0.79580883]],

       [[-1.0293318 , -0.73824903, -0.84680945],
        [-1.06575388, -0.74942007, -0.8626201 ],
        [-1.07831554, -0.75036499, -0.87113144],
        ...,
        [-0.93603355, -0.66563623, -0.72124845],
        [-0.95273954, -0.71413596, -0.75372177],
        [-0.98775576, -0.73780824, -0.79689143]],

       ...,

       [[-0.59533329, -0.68073512, -0.64476015],
        [-0

In [83]:
import numpy as np

def create_windowed_data(X, input_features, output_features, window_size, predict_steps, input_nodes, output_nodes):
    """
    创建滑动窗口数据集，允许输入和输出具有不同的节点和特征集。
    
    Args:
    X (np.array): 原始数据，形状为 (num_nodes, num_timesteps, num_features)。
    input_features (list): 输入特征的索引列表。
    output_features (list): 输出特征的索引列表。
    window_size (int): 输入窗口大小。
    predict_steps (int): 要预测的步数。
    input_nodes (list): 输入节点的索引列表。
    output_nodes (list): 输出节点的索引列表。
    
    Returns:
    np.array: X_train, 形状为 (num_samples, len(input_nodes), window_size, len(input_features))
    np.array: y_train, 形状为 (num_samples, len(output_nodes), predict_steps, len(output_features))
    """
    num_timesteps = X.shape[1]
    num_samples = num_timesteps - window_size - predict_steps + 1
    X_train = np.empty((num_samples, len(input_nodes), window_size, len(input_features)))
    y_train = np.empty((num_samples, len(output_nodes), predict_steps, len(output_features)))

    for t in range(num_samples):
        X_train[t] = X[input_nodes, t:t+window_size, :][:, :, input_features]
        y_train[t] = X[output_nodes, t+window_size:t+window_size+predict_steps, output_features][:, :, np.newaxis]  # Add a new axis to match the expected shape

    return X_train, y_train

# 设置窗口大小和预测步数
window_size = 6
predict_steps = 3

# 选择特征
input_features = [0, 1, 2]  # e.g., [0, 1, 2] could represent 'internet', 'calls', 'sms'
output_features = [0]  # e.g., [0] could represent 'internet'
input_nodes = list(range(100))  # 假设前70个节点用于输入
# output_nodes = [70, 71, 72, 73, 74, 75, 76, 77, 78, 79]  # 假设预测这10个节点
output_nodes = list(range(100))  # 假设前70个节点用于输入

# 假设 X 是从您的数据加载过程中获得的
X_train, y_train = create_windowed_data(X, input_features, output_features, window_size, predict_steps, input_nodes, output_nodes)

print("X_train shape:", X_train.shape)  # Expect: (num_samples, len(input_nodes), window_size, len(input_features))
print("y_train shape:", y_train.shape)  # Expect: (num_samples, len(output_nodes), predict_steps, len(output_features))


X_train shape: (160, 100, 6, 3)
y_train shape: (160, 100, 3, 1)


In [84]:
edge_index = []

# 因为边是无向的，所以每个连接要添加两次，一次为 (source, target)，另一次为 (target, source)
for timestep in range(num_timesteps - 6):  # 如果时间步涉及到边的创建，可能需要调整范围
    for cell_id in range(1, num_nodes):  # 假设从1开始，确保不超出边界
        if cell_id < num_nodes:  # 确保不会创建超出范围的边
            src = cell_id - 1  # CellID 从1开始的调整
            dst = cell_id     # 相邻的CellID，确保不越界

            # 添加边 (src, dst) 和 (dst, src)
            edge_index.append((src, dst))
            edge_index.append((dst, src))

# 转换为PyTorch张量
edge_index_tensor = torch.tensor(edge_index, dtype=torch.long).t().contiguous()

# 如果没有特定的边权重，可以使用全1的张量
edge_weights = torch.ones(edge_index_tensor.size(1), dtype=torch.float)


edge_index=edge_index_tensor
edge_attr=edge_weights

In [85]:
import numpy as np
from torch_geometric_temporal.signal import StaticGraphTemporalSignal

# 假设 X_train 和 y_train 已经是 numpy 数组，并且已正确形状
# 例如, X_train.shape 可能是 (num_samples, num_nodes, num_features, window_size)
# y_train.shape 可能是 (num_samples, num_nodes, predict_steps)

# 转换 X_train 和 y_train 为适合 StaticGraphTemporalSignal 的格式，另外作为numpy虽然不报错，但是不是tensor了是否合适呢
features = [X_train[i].astype(np.float32) for i in range(X_train.shape[0])]
targets = [y_train[i].astype(np.float32) for i in range(y_train.shape[0])]

# 创建 StaticGraphTemporalSignal 对象
graph_data = StaticGraphTemporalSignal(
    edge_index=edge_index_tensor, 
    edge_weight=edge_weights, 
    features=features, 
    targets=targets
)


# 数据检查

In [86]:
graph_data

In [87]:
graph_data[0]

Data(x=[100, 6, 3], edge_index=[2, 32076], edge_attr=[32076], y=[100, 3, 1])

# 模型构建及初始化

In [90]:
import torch
import torch.nn.functional as F
from torch_geometric_temporal.nn.recurrent import A3TGCN
from torch_geometric_temporal.signal import StaticGraphTemporalSignal

# 确定设备
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

# 定义TemporalGNN模型
class TemporalGNN(torch.nn.Module):
    def __init__(self, node_features, periods):
        super(TemporalGNN, self).__init__()
        # Attention Temporal Graph Convolutional Cell
        self.tgnn = A3TGCN(in_channels=window_size, 
                           out_channels=32, 
                           periods=periods)  # periods决定模型侧重短期特征还是长期特征
        # Linear layer for prediction
        self.linear = torch.nn.Linear(32, periods)  # 预测未来的值

    def forward(self, x, edge_index, edge_attr):
        """
        x = Node features for T time steps
        edge_index = Graph edge indices
        """
        h = self.tgnn(x, edge_index, edge_attr)
        h = F.relu(h)
        h = self.linear(h)
        return h

# 初始化模型
node_features = 3  # 你的节点特征维度
periods = 3  # 你要预测的时间步数
model = TemporalGNN(node_features, periods).to(device)  # 将模型移动到设备

# 定义损失函数和优化器
criterion = torch.nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

# 训练和测试

训练模型

In [94]:
# 训练模型
model.train()
epochs = 100  # 设定训练轮数
for epoch in range(epochs):
    total_loss = 0
    for time, snapshot in enumerate(graph_data):
        optimizer.zero_grad()
        
        # 将数据移动到相同的设备
        x = snapshot.x.to(device)  # [100, 6, 3]
        edge_index = snapshot.edge_index.to(device)  # [2, 32076]
        edge_attr = snapshot.edge_attr.to(device)  # [32076]
        y = snapshot.y.to(device)  # [10, 3, 1]
        
        # 模型前向传播
        output = model(x, edge_index, edge_attr)
        
        
        #选择相关站点进行预测
        output=output[70:80]
        y=y[70:80].squeeze(-1)
        
        
        
        # 只保留指定10个节点的预测输出与真实值进行对比
        loss = criterion(output, y)#output和y的维度需要完全一致
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
    
    # 打印每个epoch的损失
    print(f'Epoch {epoch+1}/{epochs}, Loss: {total_loss:.4f}')

print("Training finished!")


Epoch 1/100, Loss: 23.4206
Epoch 2/100, Loss: 13.5769
Epoch 3/100, Loss: 19.9011
Epoch 4/100, Loss: 11.8207
Epoch 5/100, Loss: 16.0301
Epoch 6/100, Loss: 12.4688
Epoch 7/100, Loss: 11.1878
Epoch 8/100, Loss: 14.0658
Epoch 9/100, Loss: 10.2636
Epoch 10/100, Loss: 10.6603
Epoch 11/100, Loss: 12.6828
Epoch 12/100, Loss: 15.8255
Epoch 13/100, Loss: 13.0236
Epoch 14/100, Loss: 12.0009
Epoch 15/100, Loss: 9.7066
Epoch 16/100, Loss: 9.3756
Epoch 17/100, Loss: 10.5370
Epoch 18/100, Loss: 13.5932
Epoch 19/100, Loss: 12.3931
Epoch 20/100, Loss: 10.6801
Epoch 21/100, Loss: 12.3870
Epoch 22/100, Loss: 10.4544
Epoch 23/100, Loss: 9.6460
Epoch 24/100, Loss: 10.6702
Epoch 25/100, Loss: 14.1176
Epoch 26/100, Loss: 14.1967
Epoch 27/100, Loss: 11.6459
Epoch 28/100, Loss: 12.3146
Epoch 29/100, Loss: 10.1089
Epoch 30/100, Loss: 10.0345
Epoch 31/100, Loss: 10.3555
Epoch 32/100, Loss: 9.6430
Epoch 33/100, Loss: 11.4576
Epoch 34/100, Loss: 11.6267
Epoch 35/100, Loss: 10.1211
Epoch 36/100, Loss: 9.9680
Epoch 

In [17]:
def test_model(model, test_loader, criterion):
    model.eval()  # 切换到评估模式
    total_loss = 0
    with torch.no_grad():  # 在测试阶段，我们不需要计算梯度
        for inputs, targets in test_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            total_loss += loss.item()
    
    average_loss = total_loss / len(test_loader)
    print(f'Test MSE: {average_loss:.4f}')

# 调用测试函数
criterion = nn.MSELoss()  # 确保使用和训练相同的损失函数
test_model(model, test_loader, criterion)


NameError: name 'test_loader' is not defined

# 层级注意力机制重构